In [1]:
import pandas as pd
import numpy as np
import cudf 
import cuml
import os

This data description, and the data that this example uses are available at 
[the UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset). 

In [2]:
RAW_DATA = os.path.expanduser('~/Data/try_this.csv')

In [3]:
# Data Description: 
#         
#         - instant: record index
#         - dteday : date
#         - season : season (1:springer, 2:summer, 3:fall, 4:winter)
#         - yr : year (0: 2011, 1:2012)
#         - mnth : month ( 1 to 12)
#         - hr : hour (0 to 23)
#         - holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
#         - weekday : day of the week
#         - workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
#         + weathersit : 
#                 - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
#                 - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
#                 - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
#                 - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
#         - temp : Normalized temperature in Celsius. The values are divided to 41 (max)
#         - atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
#         - hum: Normalized humidity. The values are divided to 100 (max)
#         - This is a good example of variables we might not have a good theoretical intuation around
#         - windspeed: Normalized wind speed. The values are divided to 67 (max)
#         - casual: count of casual users
#         - registered: count of registered users
#         - cnt: count of total rental bikes including both casual and registered

In [4]:
gdf = cudf.read_csv(RAW_DATA)

We drop the index, the timestamp (because they have broken it down for us), 
and the individual counts that make up our target variable.

In [5]:

drop_list = ['instant', 'dteday', 'casual', 'registered']
gdf = gdf.drop(drop_list)

We're going to create one-hot encoded variables, also known as dummy variables, for each of the time variables as well as the weather situation. We're going to drop one of each of these dummy variables so we don't create colinearity. 

The next data munging step we take is to convert all of our data into the same type, because that is what the cuML algorithms are expecting. 

Last, we split our data into test and train sets, training on 2011 data, and testing on 2012. 

In [6]:
dummies_list = ['season','yr', 'mnth', 'hr', 'weekday', 'weathersit']

for item in dummies_list:
    codes = gdf[item].unique()
    gdf = gdf.one_hot_encoding(item, '{}_dummy'.format(item), codes)
    gdf = gdf.drop('{}_dummy_1'.format(item))

#cuML current requires all data be of the same type, so this loop converts all values into floats
for col in gdf.columns.tolist():
    gdf[col] = gdf[col].astype('float32')
    
test = gdf.query('yr == 1').drop(dummies_list)
train = gdf.query('yr == 0').drop(dummies_list)

I am going to test out how well a small variable does against all the variables available. I select "weathersit_dummy_4" because as we see above, that seems like the worst weather for bike riding. Also, based on personal experience, bike riding in high wind is not fun either. I add workday because I'm sure it has some impact, but I'm not sure what. 

In [7]:
y_train = train['cnt']
y_test = test['cnt']

#Some of the variables, chosen by theory
X_train_1 = train[['weathersit_dummy_4', 'windspeed', 'workingday']]
X_test_1 = test[['weathersit_dummy_4', 'windspeed', 'workingday']]

#all of the varibles.
X_train_2 = train.drop('cnt')
X_test_2 = test.drop('cnt')

Here, I run two regressions. The first is based on small set of variable I think will be most impactful to bike ridership. The second regression inclueds all the variables availale.

In [8]:
OLS_1 = cuml.LinearRegression()
fit_1 = OLS_1.fit(X_train_1, y_train)
y_hat_1 = fit_1.predict(X_test_1)
MSE_1 = ((y_test - y_hat_1)**2).sum()

In [9]:
OLS_2 = cuml.LinearRegression()
fit_2 = OLS_2.fit(X_train_2, y_train)
y_hat_2 = fit_2.predict(X_test_2)
MSE_2 = ((y_test - y_hat_2)**2).sum()

In [10]:
output = {'MSE_OLS_THEORY':MSE_1,
         'MSE_OLS_ALL': MSE_2}

It looks like I was wrong, and the model with everything performs better on the out-of-sample data. 

In [11]:
print('MSE_OLS_THEORY:     {}'.format(MSE_1))
print('MSE_OLS_EVERYTHING: {}'.format(MSE_2))

MSE_OLS_THEORY:     449879008.0
MSE_OLS_EVERYTHING: 266687056.0


The "everything" model outperformed the small model, but let's see if we can do better by doing a Ridge regression. 

We're going to do a small hyperparameter search for alpha, checking 100 different values. This is fast to do with RAPIDS. Also notice that I am appending the results of each Ridge model onto the dictionary containing our earlier results, so I can more easily see which model is the best at the end. 

In [12]:
for alpha in np.arange(0.0, 1, 0.01):
    
    Ridge = cuml.Ridge(alpha=alpha, fit_intercept=True)
    fit_3 = Ridge.fit(X_train_2, y_train)
    y_hat_3 = fit_3.predict(X_test_2)
    MSE_3 = ((y_test - y_hat_3)**2).sum()
    output['MSE_RIDGE_{}'.format(alpha)] = MSE_3

Here we see that our regulaized model does better than the rest, include OLS with all the variables. 

In [13]:
print('Min MSE: {}'.format(min(output, key=output.get)))

Min MSE: MSE_RIDGE_0.1


In [14]:
#You can uncomment the code below to see all 100 model MSEs
#Notice in particular that MSE for OLS with everything and Ridge with alpha = 0 are essentially the same. 

#print(output)